# tokenについてmecabとspacy/ginzaの比較

In [1]:
from collections import Counter
import os
import re
from typing import List, Tuple, Dict, Union
import unicodedata as ud

import fire
import pandas as pd
import numpy as np
Num = Union[int, float]
STOPPOS_JP = ["形容動詞語幹", "副詞可能", "代名詞", "ナイ形容詞語幹", "特殊", "数", "接尾", "非自立"]

In [3]:
with open(os.path.expanduser("/Users/labimac/Documents/lab/ra_provisional/provisional_limco/stopwords_jp.txt"), "r") as f:
    STOPWORDS_JP = [line.strip() for line in f]
STOPPOS_JP = ["形容動詞語幹", "副詞可能", "代名詞", "ナイ形容詞語幹", "特殊", "数", "接尾", "非自立"]

with open(os.path.expanduser("/Users/labimac/Documents/lab/ra_provisional/provisional_limco/AWD-J_EX.txt"), "r") as f:
    rows = [line.strip().split("\t") for line in f]
    AWD = {word: score for word, score, _, _ in rows}
DF_jiwc = pd.read_csv(os.path.expanduser("/Users/labimac/Documents/lab/ra_provisional/provisional_limco/2017-11-JIWC.csv"), index_col=1).drop(
    columns="Unnamed: 0"
)

In [20]:
from natto import MeCab
NM = MeCab()  # NOTE: assume IPADIC
NMN = MeCab("-d /Users/labimac/Documents/lab/ra_provisional/provisional_limco/mecab-ipadic-neologd/build/mecab-ipadic-2.7.0-20070801-neologd-20200910")

In [5]:
import spacy
import ginza
NLP = spacy.load('ja_ginza')

In [108]:
text = 'ソニーのプログラミング言語とは異なり、自然言語には言葉の曖昧性が存在します。'

In [7]:
def tokens(text: str) -> np.ndarray:
    tokens = [
        (n.surface, n.feature.split(","))
        for n in NM.parse(text, as_nodes=True)
        if not n.is_eos()
    ]
    return tokens

In [8]:
tokens(text)

[('吾輩', ['名詞', '代名詞', '一般', '*', '*', '*', '吾輩', 'ワガハイ', 'ワガハイ']),
 ('は', ['助詞', '係助詞', '*', '*', '*', '*', 'は', 'ハ', 'ワ']),
 ('猫', ['名詞', '一般', '*', '*', '*', '*', '猫', 'ネコ', 'ネコ']),
 ('で', ['助動詞', '*', '*', '*', '特殊・ダ', '連用形', 'だ', 'デ', 'デ']),
 ('ある', ['助動詞', '*', '*', '*', '五段・ラ行アル', '基本形', 'ある', 'アル', 'アル']),
 ('。', ['記号', '句点', '*', '*', '*', '*', '。', '。', '。']),
 ('名前', ['名詞', '一般', '*', '*', '*', '*', '名前', 'ナマエ', 'ナマエ']),
 ('は', ['助詞', '係助詞', '*', '*', '*', '*', 'は', 'ハ', 'ワ']),
 ('まだ', ['副詞', '助詞類接続', '*', '*', '*', '*', 'まだ', 'マダ', 'マダ']),
 ('無い', ['形容詞', '自立', '*', '*', '形容詞・アウオ段', '基本形', '無い', 'ナイ', 'ナイ']),
 ('。', ['記号', '句点', '*', '*', '*', '*', '。', '。', '。'])]

In [142]:
def tokens_ginza(text: str) -> np.ndarray:
    text = text.replace('一\n\n　', '')
    doc = NLP(text)
    tokens = []
    for sent in doc.sents:
        for token in sent:
            token_tag = re.split('[,-]', token.tag_)
            token_inflection = re.split('[,-]', ginza.inflection(token))
            analysis = token_tag + token_inflection
            analysis.append(token.lemma_)
            analysis += re.split('[,-]', ginza.reading_form(token))
            tuple_ = (token.orth_, analysis)
            tokens.append(tuple_)
    return tokens

In [143]:
tokens_ginza(text)

[('吾輩', ['代名詞', '', '吾輩', 'ワガハイ']),
 ('は', ['助詞', '係助詞', '', 'は', 'ハ']),
 ('猫', ['名詞', '普通名詞', '一般', '', '猫', 'ネコ']),
 ('で', ['助動詞', '助動詞', 'ダ', '連用形', '一般', 'だ', 'デ']),
 ('ある', ['動詞', '非自立可能', '五段', 'ラ行', '終止形', '一般', 'ある', 'アル']),
 ('。', ['補助記号', '句点', '', '。', '。']),
 ('名前', ['名詞', '普通名詞', '一般', '', '名前', 'ナマエ']),
 ('は', ['助詞', '係助詞', '', 'は', 'ハ']),
 ('まだ', ['副詞', '', 'まだ', 'マダ']),
 ('無い', ['形容詞', '非自立可能', '形容詞', '終止形', '一般', '無い', 'ナイ']),
 ('。', ['補助記号', '句点', '', '。', '。'])]

In [45]:
def calc_ttrs(tokens: List[Tuple[str, List[str]]]) -> np.ndarray:
    cnt = Counter([token[1][6] for token in tokens])
    Vn = len(cnt)
    logVn = np.log(Vn)
    N = np.sum(list(cnt.values()))
    logN = np.log(N)
    return np.array(
        [
            np.divide(Vn, N),  # original plain TTR: not robust to the length
            np.divide(Vn, np.sqrt(N)),  # Guiraud's R
            np.divide(logVn, logN),  # Herdan's C_H
            np.divide(logVn, np.log(logN)),  # Rubet's k
            np.divide((logN - logVn), (logN ** 2)),  # Maas's a^2
            np.divide((1 - (Vn ** 2)), ((Vn ** 2) * logN)),  # Tuldava's LN
            np.float_power(N, np.float_power(Vn, 0.172)),  # Brunet's W
            np.divide((logN ** 2), (logN - logVn)),  # Dugast's U
        ]
    )

def measure_pos(text: str) -> np.ndarray:
    tokens = [
        (n.surface, n.feature.split(","))
        for n in NM.parse(text, as_nodes=True)
        if not n.is_eos()
    ]
    # print(tokens)

    # VERB RELATED MEASURES
    verbs = [token for token in tokens if token[1][0] == "動詞"]
    # TODO: 助動詞との連語も含める？
    # lens_verb = [len(verb) for verb in verbs]

    # CONTENT WORDS RATIO
    nouns = [token for token in tokens if token[1][0] == "名詞"]
    adjcs = [token for token in tokens if token[1][0] == "形容詞"]
    content_words = verbs + nouns + adjcs
    cwr_simple = np.divide(len(content_words), len(tokens))
    cwr_advance = np.divide(
        len(
            [
                token
                for token in content_words
                if (token[1][1] not in STOPPOS_JP) and (token[0] not in STOPWORDS_JP)
            ]
        ),
        len(tokens),
    )

    # Modifying words and verb ratio (MVR)
    advbs = [token for token in tokens if token[1][0] == "副詞"]
    padjs = [token for token in tokens if token[1][0] == "連体詞"]
    mvr = np.divide(len(adjcs + advbs + padjs), len(verbs))

    # NER
    ners = [token for token in tokens if token[1][1] == "固有名詞"]
    nerr = np.divide(len(ners), len(tokens))

    # TTR
    ttrs = calc_ttrs(tokens)

    return np.concatenate(
        (
            np.array(
                [
                    # np.mean(lens_verb),
                    # np.std(lens_verb),
                    # np.min(lens_verb),
                    # np.quantile(lens_verb, 0.25),
                    # np.median(lens_verb),
                    # np.quantile(lens_verb, 0.75),
                    # np.max(lens_verb),
                    cwr_simple,
                    cwr_advance,
                    mvr,
                    nerr,
                ]
            ),
            ttrs,
        )
    )


In [124]:
def calc_ttrs(tokens: List[Tuple[str, List[str]]]) -> np.ndarray:
    doc = NLP(text.replace('一\n\n　', ''))
    cnt = Counter([token.lemma_ for sent in doc.sents for token in sent])
    Vn = len(cnt)
    logVn = np.log(Vn)
    N = np.sum(list(cnt.values()))
    logN = np.log(N)
    return np.array(
        [
            np.divide(Vn, N),  # original plain TTR: not robust to the length
            np.divide(Vn, np.sqrt(N)),  # Guiraud's R
            np.divide(logVn, logN),  # Herdan's C_H
            np.divide(logVn, np.log(logN)),  # Rubet's k
            np.divide((logN - logVn), (logN ** 2)),  # Maas's a^2
            np.divide((1 - (Vn ** 2)), ((Vn ** 2) * logN)),  # Tuldava's LN
            np.float_power(N, np.float_power(Vn, 0.172)),  # Brunet's W
            np.divide((logN ** 2), (logN - logVn)),  # Dugast's U
        ]
    )

def measure_pos_ginza(text: str, stopwords) -> np.ndarray:
    doc = NLP(text.replace('一\n\n　', ''))
    tokens = []
    for sent in doc.sents:
        for token in sent:
            token_tag = re.split('[,-]', token.tag_) # 品詞詳細
            token_inflection = re.split('[,-]', ginza.inflection(token))  # 活用情報
            analysis = token_tag + token_inflection
            analysis.append(token.lemma_) # 基本形
            tuple_ = (token.lemma_, analysis)
            tokens.append(tuple_)
    
    verbs = [token for token in tokens if token[1][0] == "動詞"]
    nouns = [token for token in tokens if token[1][0] == "名詞"]
    adjcs = [token for token in tokens if token[1][0] == "形容詞"]
    content_words = verbs + nouns + adjcs
    cwr_simple = np.divide(len(content_words), len(tokens))
    cwr_advance = np.divide(
        len(
            [
                token
                for token in content_words
                if (token[1][1] not in STOPPOS_JP) and (token[0] not in stopwords)
            ]
        ),
        len(tokens),
    )

    advbs = [token for token in tokens if token[1][0] == "副詞"]
    padjs = [token for token in tokens if token[1][0] == "連体詞"]
    mvr = np.divide(len(adjcs + advbs + padjs), len(verbs))
    ners = [token for token in tokens if token[1][1] == "固有名詞"]
    nerr = np.divide(len(ners), len(tokens))

    ttrs = calc_ttrs(tokens)

    return np.concatenate(
        (
            np.array(
                [
                    # np.mean(lens_verb),
                    # np.std(lens_verb),
                    # np.min(lens_verb),
                    # np.quantile(lens_verb, 0.25),
                    # np.median(lens_verb),
                    # np.quantile(lens_verb, 0.75),
                    # np.max(lens_verb),
                    cwr_simple,
                    cwr_advance,
                    mvr,
                    nerr,
                ]
            ),
            ttrs,
        )
    )


In [46]:
measure_pos(text)

array([ 5.26315789e-01,  3.68421053e-01,  0.00000000e+00,  0.00000000e+00,
        8.94736842e-01,  3.90006748e+00,  9.62225186e-01,  2.62354416e+00,
        1.28292060e-02, -3.38448105e-01,  1.20705895e+02,  7.79471467e+01])

In [125]:
measure_pos_ginza(text, STOPWORDS_JP)

(array([ 5.00000000e-01,  4.00000000e-01,  0.00000000e+00,  5.00000000e-02,
         8.50000000e-01,  3.80131556e+00,  9.45749849e-01,  2.58224802e+00,
         1.81091454e-02, -3.32653155e-01,  1.31217880e+02,  5.52207173e+01]),
 [('ソニー', ['名詞', '固有名詞', '一般', '', 'ソニー']),
  ('の', ['助詞', '格助詞', '', 'の']),
  ('プログラミング', ['名詞', '普通名詞', 'サ変可能', '', 'プログラミング']),
  ('言語', ['名詞', '普通名詞', '一般', '', '言語']),
  ('と', ['助詞', '格助詞', '', 'と']),
  ('は', ['助詞', '係助詞', '', 'は']),
  ('異なる', ['動詞', '一般', '五段', 'ラ行', '連用形', '一般', '異なる']),
  ('、', ['補助記号', '読点', '', '、']),
  ('自然', ['名詞', '普通名詞', '一般', '', '自然']),
  ('言語', ['名詞', '普通名詞', '一般', '', '言語']),
  ('に', ['助詞', '格助詞', '', 'に']),
  ('は', ['助詞', '係助詞', '', 'は']),
  ('言葉', ['名詞', '普通名詞', '一般', '', '言葉']),
  ('の', ['助詞', '格助詞', '', 'の']),
  ('曖昧性', ['名詞', '普通名詞', '一般', '', '曖昧性']),
  ('が', ['助詞', '格助詞', '', 'が']),
  ('存在', ['名詞', '普通名詞', 'サ変可能', '', '存在']),
  ('する', ['動詞', '非自立可能', 'サ行変格', '連用形', '一般', 'する']),
  ('ます', ['助動詞', '助動詞', 'マス', '終止形', '一般

In [34]:
def calc_jiwc(text: str, df_jiwc) -> np.ndarray:
    tokens = [
        (n.surface, n.feature.split(","))
        for n in NM.parse(text, as_nodes=True)
        if not n.is_eos()
    ]
    jiwc_words = set(
        [token[0] if token[1][6] == "*" else token[1][6] for token in tokens]
    ) & set(df_jiwc.index)
    jiwc_vals = df_jiwc.loc[jiwc_words].sum()
    return tokens, np.divide(jiwc_vals, jiwc_vals.sum())
    # Sad Anx Anger Hate Trustful S Happy
    
print(calc_jiwc(text, DF_jiwc))

([('プログラミング', ['名詞', 'サ変接続', '*', '*', '*', '*', 'プログラミング', 'プログラミング', 'プログラミング']), ('言語', ['名詞', '一般', '*', '*', '*', '*', '言語', 'ゲンゴ', 'ゲンゴ']), ('と', ['助詞', '格助詞', '一般', '*', '*', '*', 'と', 'ト', 'ト']), ('は', ['助詞', '係助詞', '*', '*', '*', '*', 'は', 'ハ', 'ワ']), ('異なり', ['動詞', '自立', '*', '*', '五段・ラ行', '連用形', '異なる', 'コトナリ', 'コトナリ']), ('、', ['記号', '読点', '*', '*', '*', '*', '、', '、', '、']), ('自然', ['名詞', '形容動詞語幹', '*', '*', '*', '*', '自然', 'シゼン', 'シゼン']), ('言語', ['名詞', '一般', '*', '*', '*', '*', '言語', 'ゲンゴ', 'ゲンゴ']), ('に', ['助詞', '格助詞', '一般', '*', '*', '*', 'に', 'ニ', 'ニ']), ('は', ['助詞', '係助詞', '*', '*', '*', '*', 'は', 'ハ', 'ワ']), ('言葉', ['名詞', '一般', '*', '*', '*', '*', '言葉', 'コトバ', 'コトバ']), ('の', ['助詞', '連体化', '*', '*', '*', '*', 'の', 'ノ', 'ノ']), ('曖昧', ['名詞', '形容動詞語幹', '*', '*', '*', '*', '曖昧', 'アイマイ', 'アイマイ']), ('性', ['名詞', '接尾', '一般', '*', '*', '*', '性', 'セイ', 'セイ']), ('が', ['助詞', '格助詞', '一般', '*', '*', '*', 'が', 'ガ', 'ガ']), ('存在', ['名詞', 'サ変接続', '*', '*', '*', '*', '存在', 'ソンザイ', 'ソンザイ'])

In [44]:
def calc_jiwc(text: str, df_jiwc) -> np.ndarray:
    doc = NLP(text.replace('一\n\n　', ''))
    tokens = [token.lemma_ for sent in doc.sents for token in sent]

    jiwc_words = set(
        [token for token in tokens]
    ) & set(df_jiwc.index)
    jiwc_vals = df_jiwc.loc[jiwc_words].sum()
    return tokens, np.divide(jiwc_vals, jiwc_vals.sum())
    # Sad Anx Anger Hate Trustful S Happy
    
print(calc_jiwc(text, DF_jiwc))

(['プログラミング', '言語', 'と', 'は', '異なる', '、', '自然', '言語', 'に', 'は', '言葉', 'の', '曖昧性', 'が', '存在', 'する', 'ます', '。'], Sad         0.111534
Anx         0.115337
Anger       0.199913
Hate        0.139706
Trustful    0.165945
S           0.163723
Happy       0.103842
dtype: float64)


In [49]:
def measure_abst(text: str, awd) -> np.ndarray:
    tokens = [
        (n.surface, n.feature.split(","))
        for n in NMN.parse(text, as_nodes=True)
        if not n.is_eos()
    ]
    
    scores = [
        float(awd.get(token[0] if token[1][6] == "*" else token[1][6], 0))
        for token in tokens
    ]

    # top k=5 mean
    return np.array([np.mean(sorted(scores, reverse=True)[:5]), max(scores)])
print(measure_abst(text, AWD))

[2.71 3.01]


In [50]:
def measure_abst(text: str, awd) -> np.ndarray:
    doc = NLP(text.replace('一\n\n　', ''))
    tokens = [token.lemma_ for sent in doc.sents for token in sent]
    
    scores = [
        float(awd.get(token, 0))
        for token in tokens
    ]

    # top k=5 mean
    return np.array([np.mean(sorted(scores, reverse=True)[:5]), max(scores)])
print(measure_abst(text, AWD))

[2.596 2.75 ]


In [65]:
def detect_bunmatsu(text: str) -> float:
    if "\r" in text:
        sents = text.split("\r\n")
    else:
        sents = text.split("\n")
    for sent in sents:
        tokens = [
            (n.surface, n.feature.split(","))
            for n in NM.parse(text, as_nodes=True)
            if not n.is_eos()
        ]
    return tokens, tokens[-2][1][0]
print(detect_bunmatsu(text))

([('プログラミング', ['名詞', 'サ変接続', '*', '*', '*', '*', 'プログラミング', 'プログラミング', 'プログラミング']), ('言語', ['名詞', '一般', '*', '*', '*', '*', '言語', 'ゲンゴ', 'ゲンゴ']), ('と', ['助詞', '格助詞', '一般', '*', '*', '*', 'と', 'ト', 'ト']), ('は', ['助詞', '係助詞', '*', '*', '*', '*', 'は', 'ハ', 'ワ']), ('異なり', ['動詞', '自立', '*', '*', '五段・ラ行', '連用形', '異なる', 'コトナリ', 'コトナリ']), ('、', ['記号', '読点', '*', '*', '*', '*', '、', '、', '、']), ('自然', ['名詞', '形容動詞語幹', '*', '*', '*', '*', '自然', 'シゼン', 'シゼン']), ('言語', ['名詞', '一般', '*', '*', '*', '*', '言語', 'ゲンゴ', 'ゲンゴ']), ('に', ['助詞', '格助詞', '一般', '*', '*', '*', 'に', 'ニ', 'ニ']), ('は', ['助詞', '係助詞', '*', '*', '*', '*', 'は', 'ハ', 'ワ']), ('言葉', ['名詞', '一般', '*', '*', '*', '*', '言葉', 'コトバ', 'コトバ']), ('の', ['助詞', '連体化', '*', '*', '*', '*', 'の', 'ノ', 'ノ']), ('曖昧', ['名詞', '形容動詞語幹', '*', '*', '*', '*', '曖昧', 'アイマイ', 'アイマイ']), ('性', ['名詞', '接尾', '一般', '*', '*', '*', '性', 'セイ', 'セイ']), ('が', ['助詞', '格助詞', '一般', '*', '*', '*', 'が', 'ガ', 'ガ']), ('存在', ['名詞', 'サ変接続', '*', '*', '*', '*', '存在', 'ソンザイ', 'ソンザイ'])

In [70]:
def detect_bunmatsu(text: str) -> float:
    if "\r" in text:
        sents = text.split("\r\n")
    else:
        sents = text.split("\n")

    # 体言止め
    taigen = 0
    for sent in sents:
        tokens = [
            (n.surface, n.feature.split(","))
            for n in NM.parse(text, as_nodes=True)
            if not n.is_eos()
        ]
        taigen += 1 if tokens[-2][1][0] == "名詞" else 0
    ratio_taigen = np.divide(taigen, len(sents))

    return tokens, ratio_taigen
print(detect_bunmatsu(text))

([('プログラミング', ['名詞', 'サ変接続', '*', '*', '*', '*', 'プログラミング', 'プログラミング', 'プログラミング']), ('言語', ['名詞', '一般', '*', '*', '*', '*', '言語', 'ゲンゴ', 'ゲンゴ']), ('と', ['助詞', '格助詞', '一般', '*', '*', '*', 'と', 'ト', 'ト']), ('は', ['助詞', '係助詞', '*', '*', '*', '*', 'は', 'ハ', 'ワ']), ('異なり', ['動詞', '自立', '*', '*', '五段・ラ行', '連用形', '異なる', 'コトナリ', 'コトナリ']), ('、', ['記号', '読点', '*', '*', '*', '*', '、', '、', '、']), ('自然', ['名詞', '形容動詞語幹', '*', '*', '*', '*', '自然', 'シゼン', 'シゼン']), ('言語', ['名詞', '一般', '*', '*', '*', '*', '言語', 'ゲンゴ', 'ゲンゴ']), ('に', ['助詞', '格助詞', '一般', '*', '*', '*', 'に', 'ニ', 'ニ']), ('は', ['助詞', '係助詞', '*', '*', '*', '*', 'は', 'ハ', 'ワ']), ('言葉', ['名詞', '一般', '*', '*', '*', '*', '言葉', 'コトバ', 'コトバ']), ('の', ['助詞', '連体化', '*', '*', '*', '*', 'の', 'ノ', 'ノ']), ('曖昧', ['名詞', '形容動詞語幹', '*', '*', '*', '*', '曖昧', 'アイマイ', 'アイマイ']), ('性', ['名詞', '接尾', '一般', '*', '*', '*', '性', 'セイ', 'セイ']), ('が', ['助詞', '格助詞', '一般', '*', '*', '*', 'が', 'ガ', 'ガ']), ('存在', ['名詞', 'サ変接続', '*', '*', '*', '*', '存在', 'ソンザイ', 'ソンザイ'])

In [118]:
def detect_bunmatsu(text: str) -> float:
    doc = NLP(text.replace('一\n\n　', ''))
    taigen = 0
    
    for sent in doc.sents:
        tokens = []
        for token in sent:
            token_tag = re.split('[,-]', token.tag_)
            tokens.append(token_tag)
        taigen += 1 if tokens[-2][0]== "名詞" else 0
    ratio_taigen = np.divide(taigen, len([doc.sents]))

    return ratio_taigen
# print(detect_bunmatsu(text))

In [59]:
tai = '今日は晴天。'

In [119]:
detect_bunmatsu(tai)

1.0

In [ ]:
def tokens_ginza(text: str) -> np.ndarray:
    text = text.replace('一\n\n　', '')
    doc = NLP(text)
    tokens = []
    for sent in doc.sents:
        for token in sent:
            token_tag = re.split('[,-]', token.tag_)
            token_inflection = re.split('[,-]', ginza.inflection(token))
            analysis = token_tag + token_inflection
            analysis.append(token.lemma_)
            analysis += re.split('[,-]', ginza.reading_form(token))
            tuple_ = (token.orth_, analysis)
            tokens.append(tuple_)
    return tokens